# Libraries

In [1]:
import fbprophet
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from scipy.stats import boxcox
import pandas as pd
import numpy as np
from datetime import date
from datetime import time
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats
from scipy.stats import ttest_1samp
import math
import plotly 
import plotly.graph_objs as go
import plotly.express as px
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
pd.set_option('display.max_rows', None)

In [2]:
original = pd.read_csv('/Users/vladimirautier/Documents/GitHub/Week9/Project-Week-9-Final-Project/your-project/Final Project/Modified:Cleaned Data/original_values_ready_ML.csv')

In [3]:
#original.head()

I want to use the same process as in the previous notebook but on actual unforecasted data

# Using Random Forest Algorithm & Logistic Regression Algorithm

# CCI & Yield (Spread) with Copper

In [4]:
# Changing the type to datetime

original['Date'] = pd.to_datetime(original['Date']) 

In [5]:
original.head()

,Date,CCI Value,Spread,Value of Copper,NBER_Rec
0,1970-12-31,98.74387,1.390820,1029.989309,0.0
1,1971-01-31,98.92701,1.687236,1000.560900,0.0
2,1971-02-28,99.06342,2.323194,1034.447400,0.0
3,1971-03-31,99.16283,2.243524,1160.549739,0.0
4,1971-04-30,99.24342,1.877827,1240.684076,0.0


In [6]:
original.dtypes

Date               datetime64[ns]
CCI Value                 float64
Spread                    float64
Value of Copper           float64
NBER_Rec                  float64
dtype: object

In [7]:
#Set the index

original = original.resample('M', on='Date').mean()

In [8]:
original.head()

,CCI Value,Spread,Value of Copper,NBER_Rec
Date,,,,
1970-12-31,98.74387,1.390820,1029.989309,0.0
1971-01-31,98.92701,1.687236,1000.560900,0.0
1971-02-28,99.06342,2.323194,1034.447400,0.0
1971-03-31,99.16283,2.243524,1160.549739,0.0
1971-04-30,99.24342,1.877827,1240.684076,0.0


In [9]:
original.index

DatetimeIndex(['1970-12-31', '1971-01-31', '1971-02-28', '1971-03-31',
               '1971-04-30', '1971-05-31', '1971-06-30', '1971-07-31',
               '1971-08-31', '1971-09-30',
               ...
               '2019-02-28', '2019-03-31', '2019-04-30', '2019-05-31',
               '2019-06-30', '2019-07-31', '2019-08-31', '2019-09-30',
               '2019-10-31', '2019-11-30'],
              dtype='datetime64[ns]', name='Date', length=588, freq='M')

In [10]:
# Values from 1970 - 2010 to predict values from 2010-2019

In [11]:
original1 = original.loc['1970':'2006']

In [12]:
original1.tail()

,CCI Value,Spread,Value of Copper,NBER_Rec
Date,,,,
2006-08-31,100.029653,-0.212741,7729.542991,0.0
2006-09-30,100.250700,-0.216831,7622.809500,0.0
2006-10-31,100.505600,-0.321153,7507.024364,0.0
2006-11-30,100.664200,-0.471946,6994.338900,0.0
2006-12-31,100.747067,-0.418395,6656.730180,0.0


In [13]:
original2 = original.loc['2006':'2009']

In [14]:
original3 = original2.drop(['NBER_Rec'],axis=1)

In [15]:
original3.head()

,CCI Value,Spread,Value of Copper
Date,,,
2006-01-31,100.274107,0.074537,4693.583300
2006-02-28,100.252810,0.027606,4948.165600
2006-03-31,100.190963,0.094631,5090.185913
2006-04-30,100.064317,0.271242,6408.325911
2006-05-31,99.919847,0.266658,7983.889818


In [16]:
#Splitting the data

yt = original1.NBER_Rec
Xt= original1.drop('NBER_Rec', axis=1)

In [17]:
train_Xt, test_Xt, train_yt, test_yt = train_test_split(Xt, yt, test_size=0.2, stratify=yt)

# RANDOM FOREST ALGO

In [18]:
model_rf = RandomForestClassifier()

In [19]:
model_rf.fit(train_Xt,train_yt)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [20]:
rf_predictions = model_rf.predict(test_Xt) # Random forest predictions

In [21]:
rf_probs = model_rf.predict_proba(test_Xt)[:, 1]

 Anlysing the efficiency of the algorithm

In [22]:
#AUC-SCORE

In [23]:
roc_value = roc_auc_score(test_yt, rf_probs)

In [24]:
roc_value

0.9814593301435407

In [25]:
#Confusion Matrix & Accuracy

In [26]:
y_pred_t = model_rf.predict(test_Xt)

print(confusion_matrix(test_yt, y_pred_t))
print(accuracy_score(test_yt, y_pred_t))

[[76  0]
 [ 4  7]]
0.9540229885057471


In [27]:
#CROSS-VALIDATION

In [28]:
cv_results = cross_validate(model_rf, Xt, yt, cv=3)

In [29]:
sorted(cv_results.keys())

['fit_time', 'score_time', 'test_score']

In [30]:
cv_results['test_score']

array([0.74482759, 0.97916667, 0.875     ])

In [31]:
#F1 SCORE

In [32]:
f1_score(test_yt, y_pred_t)

0.7777777777777778

In [33]:
#print(rf_predictions)

 I will now use this fitted Random Forest Algorithm and use it on the rest of the values of original = original3

In [34]:
rf_predictions = model_rf.predict(original3)

In [35]:
rf_probs = model_rf.predict_proba(original3)[:, 1]

In [36]:
#print(rf_predictions)

In [37]:
type(rf_predictions)

numpy.ndarray

In [38]:
new_series = pd.Series(rf_predictions)

In [39]:
original3['NBER_Rec_Pred'] = rf_predictions

In [40]:
original3.head()

,CCI Value,Spread,Value of Copper,NBER_Rec_Pred
Date,,,,
2006-01-31,100.274107,0.074537,4693.583300,0.0
2006-02-28,100.252810,0.027606,4948.165600,0.0
2006-03-31,100.190963,0.094631,5090.185913,0.0
2006-04-30,100.064317,0.271242,6408.325911,0.0
2006-05-31,99.919847,0.266658,7983.889818,0.0


In [41]:
original3['NBER_Rec'] = original2['NBER_Rec']

In [42]:
original3.head()

,CCI Value,Spread,Value of Copper,NBER_Rec_Pred,NBER_Rec
Date,,,,,
2006-01-31,100.274107,0.074537,4693.583300,0.0,0.0
2006-02-28,100.252810,0.027606,4948.165600,0.0,0.0
2006-03-31,100.190963,0.094631,5090.185913,0.0,0.0
2006-04-30,100.064317,0.271242,6408.325911,0.0,0.0
2006-05-31,99.919847,0.266658,7983.889818,0.0,0.0


In [43]:
print(accuracy_score(original3.NBER_Rec, original3.NBER_Rec_Pred))


0.8541666666666666


# LOGISTIC REGRESSION ALGO

In [44]:
logreg = LogisticRegression()

In [45]:
logreg.fit(train_Xt,train_yt)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
y_prediction = logreg.predict(test_Xt)

 Analysing the efficiency of the algorithm

In [47]:
#AUC-SCORE

In [48]:
roc_value = roc_auc_score(test_yt, y_prediction)

In [49]:
roc_value

0.7661483253588516

In [50]:
#Confusion Matrix & Accuracy

In [51]:
print(confusion_matrix(test_yt, y_prediction))
print(accuracy_score(test_yt, y_prediction))

[[75  1]
 [ 5  6]]
0.9310344827586207


In [52]:
#CROSS-VALIDATION

In [53]:
cv_results = cross_validate(logreg , Xt, yt, cv=3)

In [54]:
sorted(cv_results.keys())

['fit_time', 'score_time', 'test_score']

In [55]:
cv_results['test_score']

array([0.5862069, 0.875    , 0.875    ])

In [56]:
#F1 SCORE

In [57]:
f1_score(test_yt, y_prediction)

0.6666666666666665

Here the Logistic Regression does not give convincing results. Random Forest seems to work the best.

# CCI & Yield (Spread) without copper

In [58]:
yield_cci = pd.read_csv('/Users/vladimirautier/Documents/GitHub/Week9/Project-Week-9-Final-Project/your-project/Final Project/Modified:Cleaned Data/original_values_yield_cci_ready_ML.csv')

In [59]:
yield_cci.head()

,Date,Spread,CCI Value,NBER_Rec
0,1970-12-31,1.390820,98.74387,0.0
1,1971-01-31,1.687236,98.92701,0.0
2,1971-02-28,2.323194,99.06342,0.0
3,1971-03-31,2.243524,99.16283,0.0
4,1971-04-30,1.877827,99.24342,0.0


In [60]:
yield_cci['Date'] = pd.to_datetime(yield_cci['Date']) 

In [61]:
#Set the index

yield_cci = yield_cci.resample('M', on='Date').mean()

In [62]:
yield_cci.head()

,Spread,CCI Value,NBER_Rec
Date,,,
1970-12-31,1.390820,98.74387,0.0
1971-01-31,1.687236,98.92701,0.0
1971-02-28,2.323194,99.06342,0.0
1971-03-31,2.243524,99.16283,0.0
1971-04-30,1.877827,99.24342,0.0


In [63]:
# Values from 1970 - 2010 to predict values from 2010-2019

In [64]:
yield_cci_1 = yield_cci.loc['1970':'2006']

In [65]:
yield_cci_1.tail()

,Spread,CCI Value,NBER_Rec
Date,,,
2006-08-31,-0.212741,100.029653,0.0
2006-09-30,-0.216831,100.250700,0.0
2006-10-31,-0.321153,100.505600,0.0
2006-11-30,-0.471946,100.664200,0.0
2006-12-31,-0.418395,100.747067,0.0


In [66]:
yield_cci_2 = yield_cci.loc['2006':'2009']

In [67]:
yield_cci_2.tail()

,Spread,CCI Value,NBER_Rec
Date,,,
2009-08-31,3.417565,98.070357,0.0
2009-09-30,3.278296,98.230083,0.0
2009-10-31,3.319015,98.298067,0.0
2009-11-30,3.349299,98.343097,0.0
2009-12-31,3.539299,98.472093,0.0


In [68]:
yield_cci_3 = yield_cci_2.drop(['NBER_Rec'],axis=1)

In [69]:
yield_cci_3.tail()

,Spread,CCI Value
Date,,
2009-08-31,3.417565,98.070357
2009-09-30,3.278296,98.230083
2009-10-31,3.319015,98.298067
2009-11-30,3.349299,98.343097
2009-12-31,3.539299,98.472093


In [70]:
#Splitting the data

yu = yield_cci_1.NBER_Rec
Xu= yield_cci_1.drop('NBER_Rec', axis=1)

In [71]:
train_Xu, test_Xu, train_yu, test_yu = train_test_split(Xu, yu, test_size=0.2, stratify=yt)

# RANDOM FOREST ALGO

In [72]:
model_rf = RandomForestClassifier()

In [73]:
model_rf.fit(train_Xu,train_yu)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [74]:
rf_predictions = model_rf.predict(test_Xu) # Random forest predictions

In [75]:
rf_probs = model_rf.predict_proba(test_Xu)[:, 1]

Analysing the efficiency of the algorithm

In [76]:
#AUC-SCORE

In [77]:
roc_value = roc_auc_score(test_yu, rf_probs)

In [78]:
roc_value

0.8409090909090908

In [79]:
#Confusion Matrix & Accuracy

In [80]:
y_pred_t = model_rf.predict(test_Xu)

print(confusion_matrix(test_yu, y_pred_t))
print(accuracy_score(test_yu, y_pred_t))

[[75  1]
 [ 4  7]]
0.9425287356321839


In [81]:
#CROSS-VALIDATION

In [82]:
cv_results = cross_validate(model_rf , Xu, yu, cv=3)

In [83]:
sorted(cv_results.keys())

['fit_time', 'score_time', 'test_score']

In [84]:
cv_results['test_score']

array([0.66896552, 0.98611111, 0.875     ])

In [85]:
#F1 SCORE

In [86]:
f1_score(test_yu, y_pred_t)

0.7368421052631579

In [87]:
#print(rf_predictions)

I will now use this fitted Random Forest Algorithm and use it on the rest of the values  = yield_curve_3

In [88]:
rf_predictions = model_rf.predict(yield_cci_3)

In [89]:
rf_probs = model_rf.predict_proba(yield_cci_3)[:, 1]

In [90]:
print(rf_predictions)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]


In [91]:
type(rf_predictions)

numpy.ndarray

In [92]:
new_series1 = pd.Series(rf_predictions)

In [93]:
yield_cci_3['NBER_Rec_Pred'] = rf_predictions

In [94]:
yield_cci_3.head()

,Spread,CCI Value,NBER_Rec_Pred
Date,,,
2006-01-31,0.074537,100.274107,0.0
2006-02-28,0.027606,100.252810,0.0
2006-03-31,0.094631,100.190963,0.0
2006-04-30,0.271242,100.064317,0.0
2006-05-31,0.266658,99.919847,0.0


In [95]:
yield_cci_3['NBER_Rec'] = yield_cci_2['NBER_Rec']

In [96]:
yield_cci_3.head()

,Spread,CCI Value,NBER_Rec_Pred,NBER_Rec
Date,,,,
2006-01-31,0.074537,100.274107,0.0,0.0
2006-02-28,0.027606,100.252810,0.0,0.0
2006-03-31,0.094631,100.190963,0.0,0.0
2006-04-30,0.271242,100.064317,0.0,0.0
2006-05-31,0.266658,99.919847,0.0,0.0


In [97]:
print(accuracy_score(yield_cci_3.NBER_Rec, yield_cci_3.NBER_Rec_Pred))

0.8125


In [98]:
#It appears that predicting with Copper seems to be more accurate